## Test

In [ ]:
def check_apt_tags_add():
    import json
    import requests
    url = "http://localhost:5000/api/tags/add/"
    headers = {'content-type': 'application/json'}
    tags = ['sender', 'he1', 'receiver', 'he2', 'message', 'We did it!']
    response = requests.post(url, json=json.dumps(tags), headers=headers)

    print(response.content)

## Progress bar

In [ ]:
def log_progress(sequence, every=10):
    from ipywidgets import IntProgress
    from IPython.display import display

    progress = IntProgress(min=0, max=len(sequence), value=0)
    display(progress)
    
    for index, record in enumerate(sequence):
        if index % every == 0:
            progress.value = index
        yield record

# Clean tags

In [ ]:
import requests    
def delete_tags(): 
    url = "http://localhost:8003/api/v1/tags/"
    r = requests.get(url)
    for t in log_progress(r.json()):
        idd = t['id']
        urld = f"http://localhost:8003/api/v1/tags/{idd}/"
        r = requests.delete(urld)     
delete_tags()

## Add tags from steps

In [ ]:
import json
import requests
import glob
files = glob.glob("../data/path/steps/*") #[0:100]
files_txt = glob.glob("../data/path/txt/*") #[0:100]

for file in log_progress(files, every=1):        
    contents = open(file).readlines()
    step = int(contents[0].strip())    
    # print(file, step)   
    text_path = files_txt[step]     
    text = open(files_txt[step-1]).read().strip().replace("\n","")
    
    url_semantic = "http://localhost:8005/api/v1/semantic/tags/"
    headers = {'content-type': 'application/json'}
    rr = requests.post(url_semantic, data = json.dumps({"text": text}), headers=headers)                
    data = rr.json()
    # if "words" in data.keys(): 
    #     words = rr.json()['words']
    # if "hrases" in data.keys(): 
    #     hrases = rr.json()['hrases']
    if "sim" in data.keys(): 
        sim = rr.json()['sim']
    # print(words, hrases, sim)
    
    for hras in sim:       
        url = "http://localhost:8003/api/v1/tags/"
        headers = {'content-type': 'application/json'}
        data = {'name': hras, "count": 0}
        response = requests.post(url, data=json.dumps(data), headers=headers)
    
    # if step > 100:
        # break